# **Chapter 9. [인터넷/웹] 최신 뉴스 이메일로 받아보기**


---
### 📝 **학습 목차**
> 9-1. 프로젝트 개요 <br>
> 9-2. 웹브라우저 실행 및 저장 - webbrowser, urllib <br>
> 9-3. 웹페이지에서 원하는 텍스트만 추출- html.parser <br>
> **9-4. 이메일 확인하기 - poplib** <br>
> 9-5. 이메일 보내기 - smtplib <br>
> 9-6. 최신 뉴스 확인하기 - BeautifulSoup <br>
> 9-7. 프로젝트 실습

## 9-4. 이메일 확인하기

> ### poplib
>  - `poplib` 는 POP3 서버에 연결하여 받은 메일을 확인하는 데 사용하는 모듈
>  - 매번 Gmail 과 같은 웹 메일에 접속해서 새로운 메일 내용 확인이 번거로울 때 이를 자동화할 경우 사용

In [3]:
import poplib

#### 이메일 계정 연결

In [46]:
# 암호화된 소켓 연결 (기본 995 포트)
server = poplib.POP3_SSL('outlook.office365.com', port=995)

In [47]:
# ID 입력
server.user('belief@postech.ac.kr')

b'+OK'

In [48]:
# Password 입력
import getpass
server.pass_(getpass.getpass())

········


b'+OK User successfully logged on.'

> #### Gmail 계정을 사용할 경우
>  - 2022년 5월 30일부터 Google은 **사용자 이름**과 **비밀번호**만 사용하여 계정에 로그인하도록 요청하는 서드 파티 앱 또는 기기의 사용을 더 이상 지원하지 않습니다. ([공지문](https://support.google.com/accounts/answer/6010255))
>  - 연동 방법
>     - ① 2단계 인증 활성화 ([설정절차](https://support.google.com/accounts/answer/185839?hl=ko&co=GENIE.Platform%3DAndroid))
>     - ② `Google 계정 관리` > `보안` > `앱 비밀번호` > `메일` 앱 선택 > `기기` (Windows 컴퓨터) > `생성`
>     - ③ 생성된 16자리 비밀번호 사용

In [4]:
# 암호화된 소켓 연결 (기본 995 포트)
server = poplib.POP3_SSL('pop.googlemail.com', port=995)

# Google ID 입력
server.user('belief.park705')

b'+OK send PASS'

In [5]:
# Password 입력
import getpass
server.pass_(getpass.getpass())

········


b'+OK Welcome.'

In [6]:
# POP3 에 저장된 메일 개수, 크기(bytes)
server.stat()

(283, 71638019)

In [7]:
# 가장 최근 메세지 번호
recent_no = server.stat()[0]

In [8]:
# 메세지 가져오기 (바이트 문자열, 두번째 요소가 메세지 내용)
server.retr(recent_no)

(b'+OK message follows',
 [b'Delivered-To: belief.park705@gmail.com',
  b'Received: by 2002:a1c:da8a:0:0:0:0:0 with SMTP id r132csp72522wmg;',
  b'        Tue, 18 Dec 2018 06:36:58 -0800 (PST)',
  b'X-Google-Smtp-Source: AFSGD/UVOqDOwNfLtsciiZv1phDRaS8i54nXvQhOpiDiuABOeXDOm6k8jenMtWjS+Sohcrf1izNA',
  b'X-Received: by 2002:a24:2104:: with SMTP id e4mr2933905ita.59.1545143818413;',
  b'        Tue, 18 Dec 2018 06:36:58 -0800 (PST)',
  b'ARC-Seal: i=1; a=rsa-sha256; t=1545143818; cv=none;',
  b'        d=google.com; s=arc-20160816;',
  b'        b=Yv32Qbi3alQSiKeuUkpvx+93G+A3pwG/OUvaq0q0rsYyXdWB0IfsjbEYRssNzrY4x5',
  b'         Tn8i8PpzdgLeADiOGXbzdS2oBZ1s6DfV+HewYk+NSnNvnyFwP0kOQGTsQenwUU2EotXr',
  b'         AieWXXLZIf9rY8GSJohB/u1Eij3vbaYpCdq+QIo9inJZtrlzyBk5y3isHOScZBh2XnSe',
  b'         u2pVT+w1xppHeHGY5f6/I6b5WHneRuPp8cKsudPONoj92exMBDVzPtD3I3Dk+yB8LbSv',
  b'         a27AHT9MqUcGKSMHwFsWDoixrFvvggyS9Iu/4Ao6r1+yycgnhT1glyEoXm9z2HcsGDFd',
  b'         dYrA==',
  b'ARC-Message-Signat

In [9]:
# 줄바꿈으로 메세지 내용 조인
raw_email = b'\n'.join(server.retr(recent_no)[1])

In [10]:
raw_email

b'Delivered-To: belief.park705@gmail.com\nReceived: by 2002:a1c:da8a:0:0:0:0:0 with SMTP id r132csp72522wmg;\n        Tue, 18 Dec 2018 06:36:58 -0800 (PST)\nX-Google-Smtp-Source: AFSGD/UVOqDOwNfLtsciiZv1phDRaS8i54nXvQhOpiDiuABOeXDOm6k8jenMtWjS+Sohcrf1izNA\nX-Received: by 2002:a24:2104:: with SMTP id e4mr2933905ita.59.1545143818413;\n        Tue, 18 Dec 2018 06:36:58 -0800 (PST)\nARC-Seal: i=1; a=rsa-sha256; t=1545143818; cv=none;\n        d=google.com; s=arc-20160816;\n        b=Yv32Qbi3alQSiKeuUkpvx+93G+A3pwG/OUvaq0q0rsYyXdWB0IfsjbEYRssNzrY4x5\n         Tn8i8PpzdgLeADiOGXbzdS2oBZ1s6DfV+HewYk+NSnNvnyFwP0kOQGTsQenwUU2EotXr\n         AieWXXLZIf9rY8GSJohB/u1Eij3vbaYpCdq+QIo9inJZtrlzyBk5y3isHOScZBh2XnSe\n         u2pVT+w1xppHeHGY5f6/I6b5WHneRuPp8cKsudPONoj92exMBDVzPtD3I3Dk+yB8LbSv\n         a27AHT9MqUcGKSMHwFsWDoixrFvvggyS9Iu/4Ao6r1+yycgnhT1glyEoXm9z2HcsGDFd\n         dYrA==\nARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;\n        h=date:message-

> ### email
>  - `email` 는 **바이트 문자열 형식의 이메일 메세지**를 사람이 알아 볼 수 있는 문자열 형태로 **파싱 및 디코딩**할 때 사용되는 모듈

In [11]:
import email
from email.header import decode_header, make_header

In [12]:
# 메세지 객체 생성
message = email.message_from_bytes(raw_email)

In [13]:
message

#### 이메일 정보 확인

In [14]:
# 송신자 확인
message.get('From')

'ResearchGate <no-reply@researchgatemail.net>'

In [15]:
# email 모듈 디코더 활용
fr = make_header(decode_header(message.get('From')))
print(fr)

ResearchGate <no-reply@researchgatemail.net>


In [16]:
# 제목 확인
subject = make_header(decode_header(message.get('Subject')))
print(subject)

Welcome to ResearchGate, Shin-Nyum


#### 이메일 본문 확인
이메일 본문을 읽어오는 것은 조금 복잡할 수 있습니다. <br>
이메일의 메시지는 **멀티파트(multipart)**로 구성될 수도 있고 아닐 수도 있기 때문인데요, 이메일에 첨부 파일이 있거나 `text/plain` 또는 `text/html` 형식의 내용을 동시에 보낸다면 **멀티파트 형식** 으로 구성됩니다. 따라서 멀티파트인지 아닌지 분기해서 확인합니다.

In [17]:
if message.is_multipart():
    # 멀티 파트라면 여러개로 나누어진 메세지를 하니씩 처리
    for part in message.walk():
        ctype = part.get_content_type()
        cdispo = str(part.get('Content-Disposition'))
        
        # 컨텐츠가 text/plain 이고, 첨부파일이 없다면
        if ctype == 'text/plain' and 'attachment' not in cdispo:
            body = part.get_payload(decode=True)  # 메세지 내용 추출
            break
else:
    # 싱글 파트라면
    body = message.get_payload(decode=True) # 메세지 내용 추출

In [18]:
body = body.decode('utf-8')

print(f"보낸사람:{fr}")
print(f"제목:{subject}")
print(f"내용:{body}")

보낸사람:ResearchGate <no-reply@researchgatemail.net>
제목:Welcome to ResearchGate, Shin-Nyum
내용:Welcome to ResearchGate, the professional network for scientists and researchers.
Get started by adding your publications to your profile to make your work visible.
67 publications in our database match your name
View publications:
https://www.researchgate.net/profile/Shin_Nyum_Park/unconfirmed?pli=1&loginT=iFmrZh_A8gh86srNCIHG-_VU-DN1RMZLbQh8eqzkaZExhbaGdwbE7wLjrmwBNN5Cms-h_IxxPslWF04vzAToBg&uid=Oj0MW13gaD76pj660zHsVuZKUXpp9CV8VYS0&cp=re41_x_p3&ch=reg&utm_medium=email&utm_source=researchgate&utm_campaign=re41&utm_term=re41_x&utm_content=re41_x_p3
-------------------------------------------------------------------------------
This message was sent to belief.park705@gmail.com by ResearchGate. To make sure you receive our updates, add ResearchGate to your address book or safe list.
See instructions: https://www.researchgate.net/help/whitelist-email?cp=re41_x_we&mfl=1&ch=reg&uid=Oj0MW13gaD76pj660zHs